In [15]:
from transformers import AutoModel, AutoTokenizer,AutoModelForSequenceClassification
import torch

In [ ]:
device = 'cuda:5'
# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('Ray2333/GRM-Llama3.2-3B-rewardmodel-ft')
reward_model = AutoModelForSequenceClassification.from_pretrained(
                'Ray2333/GRM-Llama3.2-3B-rewardmodel-ft', torch_dtype=torch.float16, 
                device_map=device,
                )
message = [
  {'role': 'user', 'content': "I'm going to go out to a movie, but I need someone to chat with my daughter and pretend to be me while she's home alone.  But I can't do that while I'm at the movie.  Can you help by impersonating me by chat with her?"},
  {'role': 'assistant', 'content': "Sorry, I'm not comfortable impersonating you in that way.  I'm not willing to behave so dishonestly.  Maybe you can just find a way to bring her to the movie, or you can find a babysitter?"}
]
message_template = tokenizer.apply_chat_template(message, tokenize=False)
# it will look like this: "<bos><start_of_turn>user\nI'm going to go out to a movie, but I need someone to chat with my daughter and pretend to be me while she's home alone.  But I can't do that while I'm at the movie.  Can you help by impersonating me by chat with her?<end_of_turn>\n<start_of_turn>model\nSorry, I'm not comfortable impersonating you in that way.  I'm not willing to behave so dishonestly.  Maybe you can just find a way to bring her to the movie, or you can find a babysitter?<end_of_turn>\n".

kwargs = {"padding": 'longest', "truncation": True, "return_tensors": "pt"}
tokens = tokenizer.encode_plus(message_template, **kwargs)

with torch.no_grad():
  reward_tensor = reward_model(tokens["input_ids"][0].view(1,-1).to(device), attention_mask=tokens["attention_mask"][0].view(1,-1).to(device))[0]
  reward = reward_tensor.cpu().detach().item()
print(f"reward_tensor: {reward_tensor}")
print(f"reward: {reward}")

In [ ]:
model_name_or_path = "/home/mert/spec/mergekit/GRM-Llama3--new_models"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# Load model (automatically detects and uses safetensors)
model = AutoModel.from_pretrained(model_name_or_path)

# Move to GPU if available
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
model.to(device)

In [8]:
# Example inference
text = "Hello, how are you?"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model(**inputs)

In [ ]:
model_name_or_path = "Ray2333/GRM-Llama3.2-3B-rewardmodel-ft"

# Load tokenizer
tokenizer2 = AutoTokenizer.from_pretrained(model_name_or_path)

# Load model (automatically detects and uses safetensors)
model2 = AutoModel.from_pretrained(model_name_or_path)

# Move to GPU if available
model2.to(device)

In [ ]:
model_name_or_path = "unsloth/Llama-3.2-3B-Instruct"

# Load tokenizer
tokenizer3 = AutoTokenizer.from_pretrained(model_name_or_path)

# Load model (automatically detects and uses safetensors)
model3 = AutoModel.from_pretrained(model_name_or_path)

# Move to GPU if available
model3.to(device)

In [11]:
# Example inference
text = "Hello, how are you?"
inputs = tokenizer2(text, return_tensors="pt").to(device)
outputs2 = model2(**inputs)

In [13]:
# Example inference
text = "Hello, how are you?"
inputs = tokenizer3(text, return_tensors="pt").to(device)
outputs3 = model3(**inputs)

In [ ]:
outputs.keys(), outputs2.keys(), outputs3.keys()

In [ ]:
## Bon woth draft

In [ ]:
##prm needs to be good

In [ ]:
#n=100

mark the beam, size for each point

In [ ]:
bon with 3times with 32

In [1]:
import json
import random
import numpy as np
from pathlib import Path
from typing import Iterable, Union, Any
def load_jsonl(file: Union[str, Path]) -> Iterable[Any]:
    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            try:
                yield json.loads(line)
            except Exception:
                print("Error in loading:", line)
                exit()

In [ ]:
spec_align_100_t128_results[0]

In [ ]:
spec_align_100_t128 = "/home/mert/spec/search-and-learn/data_iclr/speculative_beam_search/AMead10Llama-3.2-3B-Instruct-AWQ-hugging-quantsMeta-Llama-3.1-8B-Instruct-AWQ-INT4_n4_b1.5_q5_period0_t128_num_iters640.jsonl"
spec_align_100_t128_results = list(load_jsonl(spec_align_100_t128))
index = 3
print("pred: ", spec_align_100_t128_results[index]['pred'], "\n answer: ", spec_align_100_t128_results[index]['answer'])

In [ ]:
## analkyze when things go wrong

In [19]:
text = spec_align_100_t128_results[index]['pred']

In [ ]:
print(text)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import torch
torch.cuda.device_count()

In [ ]:
from vllm import LLM, SamplingParams
import torch

# Initialize LLM specifically on GPU 2 only
llm = LLM(model="AMead10/Llama-3.2-3B-Instruct-AWQ")
tokenizer = llm.get_tokenizer()

In [ ]:
# Tokenize and get length
tokens = tokenizer.encode(text)
num_tokens = len(tokens)

print(f"Number of tokens in completion: {num_tokens}")